# Section 5.4: The regularization bias, simulations


References:
- Beck, Teboule (2088) "A Fast Iterative Shrinkage-Thresholding Algorithm for Linear Inverse Problems"
- Belloni, A., Chernozhukov, V., Hansen, C. (2011) "Inference on Treatment Effects After Selection Amongst High-Dimensional Controls", https://arxiv.org/abs/1201.0224

In [1]:
from dataclasses import dataclass
import os
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import norm, multivariate_normal
import matplotlib.pyplot as plt

from ml4econometrics.ols import OLS
from ml4econometrics.penalized_regressions import Lasso

np.random.seed(99999)

FIGPATH = Path(os.getcwd()).parent / "plots"

# DGP and estimators

In [2]:
def compute_double_selection(X, d, y, s_hat):
    ols = OLS()
    if len(s_hat) > 0:
        X_ = np.c_[d, X[:, s_hat]]
    else:
        X_ = d.reshape(-1, 1)
    
    ols.fit(X_, y)
    ols.compute_std(X=X_, y=y)
    return ols.coef_[0], ols.std[1]

def double_selection_estimator(X, d, y, alpha):
    n = len(X)
    lasso = Lasso(alpha=alpha)
    ## A. Selection on treatment
    lasso.fit(X, d)
    s_d = np.nonzero(lasso.coef_)[0]

    ## B. Selection on outcome
    lasso.fit(X, y)
    s_y = np.nonzero(lasso.coef_)[0]

    ## C. Compute double selection estimator
    s_hat = np.union1d(s_y, s_d)
    if len(s_hat) > 0:
        X_ = np.c_[d, X[:, s_hat]]
    else:
        X_ = d.reshape(-1, 1)

    ols = OLS()
    ols.fit(X_, y)
    tau_hat = ols.coef_[0]
    eps_y = y - ols.predict(X_)

    if len(s_hat) > 0:
        ols.fit(X[:, s_hat], d)
        eps_d = d - ols.predict(X[:, s_hat])
    else:
        eps_d = d - d.mean()

    tau_std = np.sum((eps_d * eps_y) ** 2) / (n - len(s_hat) - 1)
    tau_std /= (np.sum(eps_d ** 2) / n) ** 2
    tau_std = np.sqrt(tau_std) / np.sqrt(n)

    return tau_hat, tau_std, s_hat

In [3]:
@dataclass
class DGP():
    """
    Generates synthetic data for illustrating the regularization bias.

    Args:
        n (int): Number of samples.
        p (int): Number of variables.
        r_y (float): R-squared for the outcome equation.
        r_d (float): R-squared for the treatment variable.
        intercept (bool): Whether to include an intercept term.
        rho (float): Covariate correlation parameter.
        tau (float): Treatment effect.

    Returns:
        tuple: A tuple containing the generated data:
            X (numpy.ndarray): Covariate matrix.
            y (numpy.ndarray): Outcome variable.
            d (numpy.ndarray): Treatment variable.
    """
    n: int = 200
    p: int = 300
    r_y: float = .1
    r_d: float = .8
    intercept: bool = True
    rho: float = .5
    tau: float = .5
        
    def __post_init__(self):
        p = self.p
        # Covariate variance matrix
        self.sigma = np.array([[self.rho ** np.abs(k - j) for k in range(p)] for j in range(p)])

        # Treatment variable coefficient
        beta = np.zeros(p)
        for j in range(p // 2):
            beta[j] = (-1) ** (j+1) / (j+1) ** 2

        # Outcome equation coefficients
        gamma = np.copy(beta)
        for j in range(p // 2 + 1, p+1):
            gamma[j-1] = (-1) ** (j + 1) / (p - j + 1) ** 2

        # Adjustment to match R-squared
        gamma *= np.sqrt(self.r_d / (1 - self.r_d) / (gamma.T @ self.sigma @ gamma))
        beta *= np.sqrt(self.r_y / (1 - self.r_y) / (beta.T @ self.sigma @ beta))
        
        self.gamma, self.beta = gamma, beta
        
        # All even-indexed covariates are dummies
        self.even = np.arange(1, p + 1) % 2 == 0
    
    def generate(self):
        # Generate covariates
        X = multivariate_normal(mean=np.zeros(self.p), cov=self.sigma).rvs(size=self.n)
        X[:, self.even] = np.where(X[:, self.even] > 0, 1, 0)

        # Treatment
        d = 1 * np.array(np.random.rand(self.n) < norm.cdf(X @ self.gamma))

        # Outcome
        y = self.tau * d + X @ self.beta + np.random.randn(self.n)

        if self.intercept:
            X = np.c_[np.ones(self.n), X]

        return X, y, d

# Parameters

In [4]:
dgp = DGP(n=200, p=300, tau=.5, intercept=False, r_y=.1, r_d=.8)
n_sim = 10_000   # nb simulations
n_folds = 5   # nb folds

# Generate random split
cvgroup = np.digitize(np.random.rand(dgp.n), np.linspace(0, 1, n_folds + 1))

# Penalty level
g = .1 / np.log(max(dgp.n, dgp.p))
c = 1.1
alpha = 2 * c * norm.ppf(1 - .5 * g / dgp.p) / np.sqrt(dgp.n) # (theoretical) Lasso penalty level

# Simulations

In [5]:
lasso_naive = Lasso(alpha=alpha, nopen=[0])
ols = OLS()

estimate, std = [], []

for b in tqdm(range(n_sim)):
    X, y, d  = dgp.generate()
    X_ = np.c_[d, X]

    # Method 1: Naive selection
    lasso_naive.fit(X_, y)
    s_hat = np.nonzero(lasso_naive.coef_[1:])[0]

    tau_naive, std_naive = compute_double_selection(X, d, y, s_hat)

    # Method 2: Double-Selection, no sample-splitting
    tau_ds, std_ds, _ = double_selection_estimator(X, d, y, alpha)

    # Method 3: with sample splitting
    tau_k, sigma_k_num, sigma_k_denom = [], [], []
    for k in range(1, n_folds + 1):
        Ik = (cvgroup == k)
        NIk = (cvgroup != k)

        g_cf = .1 / np.log(max(NIk.sum(), dgp.p))
        alpha_cf = 1.1 * norm.ppf(1 - .5 * g_cf / dgp.p) / np.sqrt(NIk.sum())

        _, _, s_hat = double_selection_estimator(X[NIk], d[NIk], y[NIk], alpha_cf)

        if len(s_hat) > 0:
            ols.fit(X[np.ix_(NIk, s_hat)], y[NIk])
            eps_y = y[Ik] - ols.predict(X[np.ix_(Ik, s_hat)])

            ols.fit(X[np.ix_(NIk, s_hat)], d[NIk])
            eps_d = d[Ik] - ols.predict(X[np.ix_(Ik, s_hat)])
        else:
            eps_y = y[Ik] - y[NIk].mean()
            eps_d = d[Ik] - d[NIk].mean()
            
        ols.fit(eps_d, eps_y)
        tau_k.append(ols.coef_[0])

        sigma_k_num.append(np.sum((eps_d*eps_y) ** 2))
        sigma_k_denom.append(np.sum(eps_d ** 2))

    sigma_k_num = np.array(sigma_k_num).sum() / len(X)
    sigma_k_denom = np.array(sigma_k_denom).sum() / len(X)

    # Save results
    estimate.append({
        'Naive post-selection': tau_naive,
        'Double selection': tau_ds,
        'Double selection w/ cross-fitting': np.array(tau_k).mean()
    })
    std.append({
        'Naive post-selection': std_naive,
        'Double selection': std_ds,
        'Double selection w/ cross-fitting': np.sqrt(sigma_k_num / sigma_k_denom ** 2) / np.sqrt(len(X))
    })
        


100%|██████████| 10000/10000 [1:33:47<00:00,  1.78it/s] 


# Results

## Table

In [6]:
estimate = pd.DataFrame(estimate)
std = pd.DataFrame(std)

table = {
    'Bias': estimate.mean() - dgp.tau,
    'RMSE': (estimate - dgp.tau).pow(2).mean().pow(.5),
    'Coverage rate': (np.abs(estimate - dgp.tau) < norm.ppf(0.975) * std).mean() # from (a+b)(a-b) < 0 iff abs(a) < b (and b > 0 always here), i.e. bounds of CI surround 0.
}
print(pd.DataFrame(table).T.round(3))


               Naive post-selection  Double selection  \
Bias                          0.397             0.012   
RMSE                          0.457             0.186   
Coverage rate                 0.212             0.942   

               Double selection w/ cross-fitting  
Bias                                       0.061  
RMSE                                       0.235  
Coverage rate                              0.915  


In [8]:
print(pd.DataFrame(table).T.round(3).to_latex())

ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

## Histograms

In [13]:
plt.rcParams.update({'font.size': 22})
estimate_ = (estimate - dgp.tau) / std
fig, axs = plt.subplots(1, len(estimate_.columns), figsize=(20, 6), sharex=True, sharey=True, dpi=800)
axs = axs.ravel()

x = np.linspace(-1.1 * np.abs(estimate_).max().max(), 1.1 * np.abs(estimate_).max().max(), 100)

for ax, (title, data) in zip(axs, estimate_.items()):
    # Plot histogram with light grey color
    ax.hist(data, bins=int(n_sim ** (1 / 2)), color='lightgrey', density=True)
    ax.plot(x, norm.pdf(x, 0, 1), color='black', linestyle='dashed', linewidth=2)
    
    # Set plot title and labels
    ax.set_title(title)
    ax.set_xlabel(r'$\sqrt{n}(\widehat \tau - \tau_0) / \widehat \sigma$')
    ax.set_ylabel('Frequency')
    ax.yaxis.set_ticks(np.arange(0, .5, .1))

plt.tight_layout()
plt.savefig(
    FIGPATH / "Fig_5_1.jpg",
    dpi=300,
    bbox_inches='tight'
    )